# Deep Learning

**Enunciado:** <br>
Compare o desempenho da DBN proposta por Hinton e da CNN Lenet-5 proposta por LeCun, sobre o dataset MNIST. <br>
Verifiquem a precisão/acurácia e o tempo de aprendizado e processamento.<br>

**Introdução:**
O objetivo desta atividade é verificar e comparar o desempenho de modelos já prontos. A referência aos sites utilizados será realizada conforme introdução de cada tópico.

**Sobre o dataset:**
O Dataset MNIST possui milhares de imagens manuscritas dos dígitos de 0-9.

## DBN

Para treinar uma DBN solucionando o problema dos dígitos MNIST, foi utilizado o código disponível no GitHub:<br>
https://github.com/albertbup/deep-belief-network <br>
    Todavia, foi adicionada a biblioteca **time** para contagem do tempo de aprendizado e processamento.<br>

Obs: Caso copiado e colado o código exatamente conforme fonte, o seguinte erro nos é aprensentado: 
"AttributeError: module 'tensorflow' has no attribute 'Session'".
Para soluciona-lo, a importação "from dbn.tensorflow import SupervisedDBNClassification" foi substituída pelo equivalente "from dbn.models import SupervisedDBNClassification" 

In [1]:
import numpy as np
import time #library to count the time

np.random.seed(1337)  # for reproducibility
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics.classification import accuracy_score
from dbn.models import SupervisedDBNClassification


total_start_time = time.time()

# Loading dataset
digits = load_digits()
X, Y = digits.data, digits.target

# Data scaling
X = (X / 16).astype(np.float32)

# Splitting data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

#Adicionando a biblioteca time para análise de desempenho
start_time = time.time()

# Training
classifier = SupervisedDBNClassification(hidden_layers_structure=[256, 256],
                                         learning_rate_rbm=0.05,
                                         learning_rate=0.1,
                                         n_epochs_rbm=10,
                                         n_iter_backprop=100,
                                         batch_size=32,
                                         activation_function='relu',
                                         dropout_p=0.2)
classifier.fit(X_train, Y_train)
#Tempo de término
end_time = time.time()
print("TEMPO DE TREINO = ", end_time - start_time)

# Save the model using pkl
classifier.save('model.pkl')
# Restore the model
classifier = SupervisedDBNClassification.load('model.pkl')

# Test
#Tempo de inicio
start_time = time.time()

Y_pred = classifier.predict(X_test)
print('Done.\nAccuracy: %f' % accuracy_score(Y_test, Y_pred))

#Adicionando a biblioteca time para análise de desempenh
end_time = time.time()
print("TEMPO DE CLASSIFICAÇÃO = ", end_time - start_time)

total_end_time = time.time()
print("TEMPO DE PROCESSAMENTO = ", total_end_time - total_start_time)

c:\users\letic\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


[START] Pre-training step:
>> Epoch 1 finished 	RBM Reconstruction error 2.740666
>> Epoch 2 finished 	RBM Reconstruction error 1.805594
>> Epoch 3 finished 	RBM Reconstruction error 1.432523
>> Epoch 4 finished 	RBM Reconstruction error 1.274614
>> Epoch 5 finished 	RBM Reconstruction error 1.127715
>> Epoch 6 finished 	RBM Reconstruction error 1.016233
>> Epoch 7 finished 	RBM Reconstruction error 0.931797
>> Epoch 8 finished 	RBM Reconstruction error 0.904623
>> Epoch 9 finished 	RBM Reconstruction error 0.891258
>> Epoch 10 finished 	RBM Reconstruction error 0.790757
>> Epoch 1 finished 	RBM Reconstruction error 2.880171
>> Epoch 2 finished 	RBM Reconstruction error 1.491411
>> Epoch 3 finished 	RBM Reconstruction error 1.120919
>> Epoch 4 finished 	RBM Reconstruction error 0.921536
>> Epoch 5 finished 	RBM Reconstruction error 0.763747
>> Epoch 6 finished 	RBM Reconstruction error 0.667087
>> Epoch 7 finished 	RBM Reconstruction error 0.573752
>> Epoch 8 finished 	RBM Reconstructi

**Overview DBN:**

- Acurácia = 0.997222

- Tempo de execução no TREINO = 135.22251749038696 s

- Tempo de execução no CLASSIFICAÇÃO = 0.0059850215911865234 s

- Tempo de processamento TOTAL = 135.33073830604553 s

## CNN Lenet-5

Para treinar uma DBN solucionando o problema dos dígitos MNIST, foi utilizado o código disponível no site: <br>
https://medium.com/@mgazar/lenet-5-in-9-lines-of-code-using-keras-ac99294c8086 <br>
<br>
Este código foi adaptado para a checagem do tempo de processamento.

In [2]:
#LeNet-5
import gzip
import numpy as np
import pandas as pd
from time import time

from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
import keras.layers as layers
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.callbacks import TensorBoard

from requests import get

def download_file(url, file_name):
    with open(file_name, "wb") as file:
        response = get(url)
        file.write(response.content)


def read_mnist(images_path: str, labels_path: str):
    with gzip.open(labels_path, 'rb') as labelsFile:
        labels = np.frombuffer(labelsFile.read(), dtype=np.uint8, offset=8)

    with gzip.open(images_path,'rb') as imagesFile:
        length = len(labels)
        # Load flat 28x28 px images (784 px), and convert them to 28x28 px
        features = np.frombuffer(imagesFile.read(), dtype=np.uint8, offset=16) \
                        .reshape(length, 784) \
                        .reshape(length, 28, 28, 1)
        
    return features, labels

download_file('http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz', 'train-images-idx3-ubyte.gz')
download_file('http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz', 'train-labels-idx1-ubyte.gz')
download_file('http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz', 't10k-images-idx3-ubyte.gz')
download_file('http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz', 't10k-labels-idx1-ubyte.gz')


model = keras.Sequential()
model.add(layers.Conv2D(filters=6, kernel_size=(3, 3),
activation='relu', input_shape=(32,32,1)))
model.add(layers.AveragePooling2D())
model.add(layers.Conv2D(filters=16, kernel_size=(3, 3),
activation='relu'))
model.add(layers.AveragePooling2D())
model.add(layers.Flatten())
model.add(layers.Dense(units=120, activation='relu'))
model.add(layers.Dense(units=84, activation='relu'))
model.add(layers.Dense(units=10, activation = 'softmax'))

model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

#Apply at Mnist
from requests import get

def download_file(url, file_name):
    with open(file_name, "wb") as file:
        response = get(url)
        file.write(response.content)

download_file('http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz', 'train-images-idx3-ubyte.gz')
download_file('http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz', 'train-labels-idx1-ubyte.gz')
download_file('http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz', 't10k-images-idx3-ubyte.gz')
download_file('http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz', 't10k-labels-idx1-ubyte.gz')

# Commented out IPython magic to ensure Python compatibility.
import gzip
import numpy as np
import pandas as pd
from time import time

from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
import keras.layers as layers
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.callbacks import TensorBoard


total_start_time = time()

def read_mnist(images_path: str, labels_path: str):
    with gzip.open(labels_path, 'rb') as labelsFile:
        labels = np.frombuffer(labelsFile.read(), dtype=np.uint8, offset=8)

    with gzip.open(images_path,'rb') as imagesFile:
        length = len(labels)
        # Load flat 28x28 px images (784 px), and convert them to 28x28 px
        features = np.frombuffer(imagesFile.read(), dtype=np.uint8, offset=16) \
                        .reshape(length, 784) \
                        .reshape(length, 28, 28, 1)
        
    return features, labels

train = {}
test = {}

train['features'], train['labels'] = read_mnist('train-images-idx3-ubyte.gz', 'train-labels-idx1-ubyte.gz')
test['features'], test['labels'] = read_mnist('t10k-images-idx3-ubyte.gz', 't10k-labels-idx1-ubyte.gz')

"""### Explore the data
It is always a good to do some data exploration before we start using it, find outliers, and decide if we need a preprocessing phase to uniform or augment it. And also to make sure that all the classes are covered by or more or less the same number of samples.
"""

print('# of training images:', train['features'].shape[0])
print('# of test images:', test['features'].shape[0])

"""#### Display some images"""

train_labels_count = np.unique(train['labels'], return_counts=True)
dataframe_train_labels = pd.DataFrame({'Label':train_labels_count[0], 'Count':train_labels_count[1]})

"""### Split training data into training and validation"""

validation = {}
train['features'], validation['features'], train['labels'], validation['labels'] = train_test_split(train['features'], train['labels'], test_size=0.2, random_state=0)

print('# of training images:', train['features'].shape[0])
print('# of validation images:', validation['features'].shape[0])

"""## 2. Prepare our input features
The LeNet architecture accepts a 32x32 pixel images as input, mnist data is 28x28 pixels. We simply pad the images with zeros to overcome that.
"""

# Pad images with 0s
train['features']      = np.pad(train['features'], ((0,0),(2,2),(2,2),(0,0)), 'constant')
validation['features'] = np.pad(validation['features'], ((0,0),(2,2),(2,2),(0,0)), 'constant')
test['features']       = np.pad(test['features'], ((0,0),(2,2),(2,2),(0,0)), 'constant')
    
print("Updated Image Shape: {}".format(train['features'][0].shape))

model = keras.Sequential()

model.add(layers.Conv2D(filters=6, kernel_size=(3, 3), activation='relu', input_shape=(32,32,1)))
model.add(layers.AveragePooling2D())

model.add(layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu'))
model.add(layers.AveragePooling2D())

model.add(layers.Flatten())

model.add(layers.Dense(units=120, activation='relu'))

model.add(layers.Dense(units=84, activation='relu'))

model.add(layers.Dense(units=10, activation = 'softmax'))

model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

EPOCHS = 10
BATCH_SIZE = 128

X_train, y_train = train['features'], to_categorical(train['labels'])
X_validation, y_validation = validation['features'], to_categorical(validation['labels'])

train_generator = ImageDataGenerator().flow(X_train, y_train, batch_size=BATCH_SIZE)
validation_generator = ImageDataGenerator().flow(X_validation, y_validation, batch_size=BATCH_SIZE)

print('# of training images:', train['features'].shape[0])
print('# of validation images:', validation['features'].shape[0])

steps_per_epoch = X_train.shape[0]//BATCH_SIZE
validation_steps = X_validation.shape[0]//BATCH_SIZE

tensorboard = TensorBoard(log_dir="logs/{}".format(time()))

#Medição do tempo
start_time = time()

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=EPOCHS, 
                    validation_data=validation_generator, validation_steps=validation_steps, 
                    shuffle=True, callbacks=[tensorboard])

end_time = time()
print("TEMPO DE TREINO =", end_time - start_time)


start_time = time()

score = model.evaluate(test['features'], to_categorical(test['labels']))
print('Test accuracy:', score[1])

end_time = time()
print("TEMPO DE CLASSIFICAÇÃO = ", end_time - start_time)

total_end_time = time()
print("TEMPO DE PROCESSAMENTO = ", total_end_time - total_start_time)

# of training images: 60000
# of test images: 10000
# of training images: 48000
# of validation images: 12000
Updated Image Shape: (32, 32, 1)
# of training images: 48000
# of validation images: 12000
Epoch 1/10


c:\users\letic\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


375/375 [==============================] - 17s 43ms/step - loss: 1.0464 - accuracy: 0.8238 - val_loss: 0.1002 - val_accuracy: 0.9693
Epoch 2/10
375/375 [==============================] - 14s 37ms/step - loss: 0.0842 - accuracy: 0.9739 - val_loss: 0.0725 - val_accuracy: 0.9778
Epoch 3/10
375/375 [==============================] - 14s 37ms/step - loss: 0.0559 - accuracy: 0.9824 - val_loss: 0.0601 - val_accuracy: 0.9815
Epoch 4/10
375/375 [==============================] - 14s 38ms/step - loss: 0.0435 - accuracy: 0.9858 - val_loss: 0.0533 - val_accuracy: 0.9835
Epoch 5/10
375/375 [==============================] - 14s 37ms/step - loss: 0.0338 - accuracy: 0.9891 - val_loss: 0.0591 - val_accuracy: 0.9826
Epoch 6/10
375/375 [==============================] - 14s 38ms/step - loss: 0.0286 - accuracy: 0.9901 - val_loss: 0.0467 - val_accuracy: 0.9858
Epoch 7/10
375/375 [==============================] - 14s 38ms/step - loss: 0.0235 - accuracy: 0.9923 - val_loss: 0.0539 - val_accuracy: 0.9841
Epo

**Overview Lenet-5:**

- Acurácia = 0.9790999889373779

- Tempo de execução no TREINO = 148.50358033180237 s

- Tempo de execução no CLASSIFICAÇÃO = 2.331021547317505 s

- Tempo de processamento TOTAL = 151.50507521629333 s

## Conclusão comparativa

- Acurácia:<br>
Lenet-5 = 0.9790999889373779 <br>
DBN = 0.997222<br>
<br>
- Tempo de execução no TREINAMENTO: <br>
Lenet-5 = 148.50358033180237 s<br>
DBN = 135.22251749038696 s<br>
<br>
- Tempo de execução no CLASSIFICAÇÃO:<br>
Lenet-5 =  2.331021547317505 s<br>
DBN = 0.0059850215911865234 s<br>
 <br>   
- Tempo de processamento TOTAL:<br>
Lenet-5 = 151.50507521629333 s<br>
DBN = 135.33073830604553 s<br>

Nota-se portanto que o modelo DBN, proposto por Hilton, apresentou uma acurácia maior e um tempo de processamento total menor, comparado ao Lenet-5 de LeCun.
